<a href="https://colab.research.google.com/github/ChessEngineUS/hybrid-quantum-protein-folding/blob/main/notebooks/colab_quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧬 Hybrid Quantum-AI Protein Folding - Quickstart

This notebook demonstrates the basic usage of the hybrid quantum-classical protein folding framework.

**What you'll learn:**
- 📦 Install the framework
- 🧬 Predict protein structures
- 📊 Visualize results in 3D
- ⚛️ Use quantum simulators

**Runtime:** T4 GPU (recommended) or CPU

**⚠️ Note:** After installation, you may need to restart the runtime (Runtime > Restart runtime) for all imports to work correctly.

## 📦 Setup and Installation

First, let's install the framework and its dependencies.

In [ ]:
import sys
import os

# Remove old installations
if os.path.exists('hybrid-quantum-protein-folding'):
    !rm -rf hybrid-quantum-protein-folding

# Clone repository
!git clone https://github.com/ChessEngineUS/hybrid-quantum-protein-folding.git

# Change to repository directory
%cd hybrid-quantum-protein-folding

# Install dependencies (using Colab-optimized requirements)
!pip install -q -r requirements-colab.txt

# Install package in development mode
!pip install -q -e .

print('✅ Installation complete!')
print('\n⚠️ If you see import errors below:')
print('   1. Go to Runtime > Restart runtime')
print('   2. Skip this cell and run from \'Imports\' onwards')

## 📥 Imports

Import the necessary modules.

**If this cell fails with ImportError**, please restart the runtime (Runtime > Restart runtime) and run this cell again.

In [ ]:
# Force reload modules to get latest version
import sys
import importlib

# Remove cached imports
modules_to_clear = [k for k in list(sys.modules.keys()) if k.startswith('hqpf')]
for module in modules_to_clear:
    del sys.modules[module]

import torch
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Change to repo directory if not already there
import os
if not os.path.exists('hqpf'):
    os.chdir('/content/hybrid-quantum-protein-folding')

# Now import the model
from hqpf.models import HybridModel
from hqpf.data.benchmarks import load_benchmark_protein

print('✅ Imports successful!')
print(f'PyTorch: {torch.__version__}')
print(f'CUDA available: {torch.cuda.is_available()}')
print(f'Device: {"cuda" if torch.cuda.is_available() else "cpu"}')

## 🧬 Predict Protein Structure

Let's predict the structure of a 10-residue peptide.

In [ ]:
# Define sequence
sequence_str = "ACDEFGHIKL"

# Amino acid to index mapping
aa_to_idx = {
    'A': 0, 'R': 1, 'N': 2, 'D': 3, 'C': 4,
    'Q': 5, 'E': 6, 'G': 7, 'H': 8, 'I': 9,
    'L': 10, 'K': 11, 'M': 12, 'F': 13, 'P': 14,
    'S': 15, 'T': 16, 'W': 17, 'Y': 18, 'V': 19
}

# Determine device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'💻 Using device: {device}')

# Convert to tensor and move to device (IMPORTANT!)
sequence = torch.tensor(
    [aa_to_idx[aa] for aa in sequence_str], 
    dtype=torch.long, 
    device=device
)

print(f'\n🧬 Sequence: {sequence_str}')
print(f'Length: {len(sequence)} residues')
print(f'Tensor device: {sequence.device}')

# Initialize model
model = HybridModel(
    n_residues=len(sequence),
    n_qubits=10,
    use_quantum=False,  # Use quantum simulator
    device=device
)

print('\n✅ Model initialized!')

# Predict structure
print('\n🔬 Running prediction...')
model.eval()
with torch.no_grad():
    outputs = model(sequence, n_candidates=10)

best_structure = outputs['best_structure']
best_energy = outputs['best_energy']

print(f'\n✅ Prediction complete!')
print(f'⚡ Best energy: {best_energy.item():.4f}')
print(f'📊 Structure shape: {best_structure.shape}')
print(f'⚛️ Quantum fraction: {outputs["quantum_fraction"]:.2%}')

## 📊 Visualize Structure

Let's create a 3D visualization of the predicted structure.

In [ ]:
# Convert to numpy
structure_np = best_structure.cpu().numpy()

# Create 3D plot
fig = plt.figure(figsize=(12, 10))
ax = fig.add_subplot(111, projection='3d')

# Plot backbone
ax.plot(structure_np[:, 0], structure_np[:, 1], structure_np[:, 2],
        'b-', linewidth=3, alpha=0.6, label='Backbone')

# Plot residues
scatter = ax.scatter(structure_np[:, 0], structure_np[:, 1], structure_np[:, 2],
                    c=range(len(sequence)), cmap='rainbow', s=200, alpha=0.8,
                    edgecolors='black', linewidths=2)

# Add labels
for i, (x, y, z) in enumerate(structure_np):
    ax.text(x, y, z, sequence_str[i], fontsize=14, fontweight='bold')

# Styling
ax.set_xlabel('X (Å)', fontsize=14, fontweight='bold')
ax.set_ylabel('Y (Å)', fontsize=14, fontweight='bold')
ax.set_zlabel('Z (Å)', fontsize=14, fontweight='bold')
ax.set_title(f'Predicted Structure: {sequence_str}\nEnergy: {best_energy.item():.4f}',
            fontsize=16, fontweight='bold')
ax.legend(fontsize=12)
plt.colorbar(scatter, label='Residue Index', pad=0.1)

plt.tight_layout()
plt.show()

print('✅ Visualization complete! 🧬')

## 🧪 Try a Benchmark Protein

Now let's try one of the built-in benchmark proteins.

In [ ]:
from hqpf.data.benchmarks import list_benchmark_proteins, AA_TO_IDX

# List available test proteins
print('🧪 Available test proteins:')
for protein_id in list_benchmark_proteins('test'):
    protein = load_benchmark_protein(protein_id, 'test')
    print(f'  - {protein_id}: {protein.sequence} ({len(protein.sequence)} residues)')

# Load a helix
print('\n🔬 Predicting alpha helix...')
helix_protein = load_benchmark_protein('helix_12', 'test')
helix_sequence = helix_protein.to_tensor(AA_TO_IDX, device=device)

# Predict
model_helix = HybridModel(
    n_residues=len(helix_sequence),
    n_qubits=12,
    use_quantum=False,
    device=device
)

model_helix.eval()
with torch.no_grad():
    helix_outputs = model_helix(helix_sequence, n_candidates=10)

print(f'✅ Helix prediction complete!')
print(f'⚡ Energy: {helix_outputs["best_energy"].item():.4f}')
print(f'🧬 Secondary structure: {helix_protein.secondary_structure}')

## 🚀 Next Steps

Congratulations! You've successfully predicted and visualized protein structures.

**Try these next:**
- 🧬 Experiment with different sequences
- 📊 Increase `n_candidates` for better predictions (e.g., 50)
- ⚛️ Enable quantum evaluation with `use_quantum=True`
- 📊 Try mini-proteins from the benchmark dataset
- 💾 Train your own model on custom data

**Resources:**
- 📚 [Documentation](https://github.com/ChessEngineUS/hybrid-quantum-protein-folding/tree/main/docs)
- 📦 [Examples](https://github.com/ChessEngineUS/hybrid-quantum-protein-folding/tree/main/examples)
- 🐛 [GitHub Repository](https://github.com/ChessEngineUS/hybrid-quantum-protein-folding)
- ❓ [Troubleshooting Guide](https://github.com/ChessEngineUS/hybrid-quantum-protein-folding/blob/main/notebooks/COLAB_INSTRUCTIONS.md)

**Performance Tips:**
- 🚀 Use T4 GPU for 5-10x speedup
- 💾 Reduce `n_candidates` to 5 if running out of memory
- ⚛️ Quantum simulation is slow - start with `use_quantum=False`
- 📊 Use surrogate model for fast energy evaluation